In [1]:
pip install nevergrad

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
import nevergrad as ng
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('x_train_preprocessed.csv', header=0)
x_train_preprocessed = df.copy()
x_train_preprocessed= np.array(x_train_preprocessed)

In [3]:
y_train = pd.read_csv('y_train.csv', index_col=['id'])
labels_raw = y_train.copy() # a copy of the raw labels
labels_sorted = y_train.copy().sort_values(by = ['y']) # a copy of the raw labels sorted
#labels_raw.head()

In [4]:
xtest = pd.read_csv('x_test_preprocessed.csv')
x_test_preprocessed = xtest.copy()


In [5]:
data = np.array(df)
l = np.array(y_train)
labels = [l[i][0] for i in range(len(l))]


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import nevergrad as ng
from sklearn.metrics import f1_score



X_train, X_test, y_train, y_test = train_test_split(x_train_preprocessed, labels, test_size=0.2, random_state=42)

# Define the objective function for Nevergrad optimization
def objective_function(hyperparameters):
    # Create and train the LGBM model
    model = RandomForestClassifier(
        n_estimators=int(hyperparameters["n_estimators"]),
        criterion=hyperparameters["criterion"],
        max_leaf_nodes=hyperparameters["max_leaf_nodes"],
        max_depth=int(hyperparameters["max_depth"]),
        max_features=hyperparameters["max_features"],
        min_samples_split=hyperparameters["min_samples_split"],
        min_samples_leaf=hyperparameters["min_samples_leaf"],
        min_weight_fraction_leaf=hyperparameters["min_weight_fraction_leaf"],
        min_impurity_decrease=hyperparameters["min_impurity_decrease"],
        random_state=hyperparameters["random_state"],
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    # Use accuracy as the performance metric
    return -f1_score(y_test, preds, average='micro')  # Negative because Nevergrad minimizes the objective function

# Set up the Nevergrad optimizer
parametrization = ng.p.Dict(
    n_estimators=ng.p.Scalar(lower=100, upper=1000).set_integer_casting(),
    criterion=ng.p.Choice(["gini", "entropy", 'log_loss']),
    max_leaf_nodes=ng.p.Scalar(lower=10, upper=100).set_integer_casting(),
    max_depth=ng.p.Scalar(lower=1, upper=50).set_integer_casting(),
    max_features=ng.p.Choice(['sqrt', 'log2']),
    min_samples_split=ng.p.Scalar(lower=2, upper=10).set_integer_casting(),
    min_samples_leaf=ng.p.Scalar(lower=1, upper=10).set_integer_casting(),
    min_weight_fraction_leaf=ng.p.Log(lower=0.0001, upper=0.5),
    min_impurity_decrease=ng.p.Log(init=0),
    random_state=ng.p.Scalar(lower=0, upper=100).set_integer_casting()
)
optimizer = ng.optimizers.OnePlusOne(parametrization=parametrization, budget=100)

# Run the optimization
recommendation = optimizer.minimize(objective_function)

# Train the final model with optimized hyperparameters
best_hyperparams = recommendation.value
final_model = RandomForestClassifier(**best_hyperparams)
final_model.fit(X_train, y_train)

# Now, final_model is your trained model with optimized hyperparameters

/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/nevergrad/parametrization/_datalayers.py:195: RuntimeWarning: divide by zero encountered in log
  return np.log(value) / np.log(self._base)
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/nevergrad/parametrization/data.py:309: RuntimeWarning: invalid value encountered in subtract
  return reference._to_reduced_space(self._value - reference._value)
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/nevergrad/parametrization/data.py:309: RuntimeWarning: invalid value encountered in subtract
  return reference._to_reduced_space(self._value - reference._value)
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/nevergrad/parametrization/data.py:309: RuntimeWarning: invalid value encountered in subtract
  return reference._to_reduced_space(self._value - reference._value)
/home/linuxbrew/.linuxbrew/Cellar/python@3.11/

RandomForestClassifier(criterion='log_loss', max_depth=22, max_leaf_nodes=39,
                       min_samples_leaf=6, min_samples_split=6,
                       min_weight_fraction_leaf=0.00020461588905060978,
                       n_estimators=805, random_state=87)

In [7]:
# Predict the test set
y_pred = final_model.predict(X_test)
f1 = f1_score(y_test, y_pred, average='micro')
f1

0.822265625

In [8]:
print("Meilleurs hyperparamètres trouvés:", best_hyperparams)

Meilleurs hyperparamètres trouvés: {'n_estimators': 805, 'criterion': 'log_loss', 'max_leaf_nodes': 39, 'max_depth': 22, 'max_features': 'sqrt', 'min_samples_split': 6, 'min_samples_leaf': 6, 'min_weight_fraction_leaf': 0.00020461588905060978, 'min_impurity_decrease': 0.0, 'random_state': 87}


In [9]:
print(type(x_test_preprocessed))

<class 'pandas.core.frame.DataFrame'>


In [10]:
x_test_preprocessed.loc

In [41]:

# Assuming final_model is already trained
y_pred = final_model.predict(x_test_preprocessed)

# Creating a DataFrame to store results
out = pd.DataFrame(index=x_test_preprocessed.index, columns=['id', 'y'])
out['id'] = x_test_preprocessed.iloc[:, 0].astype(int)
  # Assuming 'id' is at index 0 in x_test_preprocessed and converting to int
out['y'] = y_pred

# Change the first element in the 'id' column to 'Id'
out.at[out.index[0], 'id'] = 'Id'

# Saving the DataFrame to a CSV file
out.to_csv("out.csv", index=False)  # Set index=False to exclude row indices in the CSV file


/Users/youssefraoui/miniconda3/envs/Test/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
